<a href="https://colab.research.google.com/github/madziejm/1e100-ibu/blob/master/1e100ibu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Dependencies siorbing

In [ ]:
import torchtext
import io
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data import get_tokenizer

import torch

dev = 'cuda' if torch.cuda.is_available else 'cpu'
# dev = 'cpu'
print(f'dev = {dev}')

dev = cuda


In [ ]:
!pip install icecream
from icecream import ic

#### Dataset siorbing

In [ ]:
RATEBEER_FILE = '/content/SNAP-Ratebeer.txt'
! export RATEBEER_FILE='/content/SNAP-Ratebeer.txt'
! [ -e $RATEBEER_FILE ] || gdown --id '12tEEYQcHZtg5aWyfIiWWVIDAJNT-5d_T' # https://drive.google.com/file/d/12tEEYQcHZtg5aWyfIiWWVIDAJNT-5d_T/view?usp=sharing
! echo "Dataset head (trailing newline makes entry end): "
! head -n 16 $RATEBEER_FILE
! iconv -f ISO-8859-1 -t UTF-8 $RATEBEER_FILE -o {RATEBEER_FILE}.new && mv {RATEBEER_FILE}.new $RATEBEER_FILE

Downloading...
From: https://drive.google.com/uc?id=12tEEYQcHZtg5aWyfIiWWVIDAJNT-5d_T
To: /content/SNAP-Ratebeer.txt
100% 1.74G/1.74G [00:18<00:00, 93.4MB/s]
Dataset head (trailing newline makes entry end): 
beer/name: John Harvards Simcoe IPA
beer/beerId: 63836
beer/brewerId: 8481
beer/ABV: 5.4
beer/style: India Pale Ale &#40;IPA&#41;
review/appearance: 4/5
review/aroma: 6/10
review/palate: 3/5
review/taste: 6/10
review/overall: 13/20
review/time: 1157587200
review/profileName: hopdog
review/text: On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.

beer/name: John Harvards Simcoe IPA
beer/beerId: 63836


### Vocab representation

In [ ]:
UNKNOWN_TOKEN = "<unk>"

tokenizer = get_tokenizer('basic_english')

In [ ]:
MAX_REVIEWS=10
def create_vocab(dataset_path, max_reviews=float('inf')):
  def yield_tokens():
    review_count = 0
    with io.open(dataset_path, encoding='utf-8') as f:
      for line in f:
        review_text_prefix = 'review/text: '
        if line.startswith(review_text_prefix):
          line = line[len(review_text_prefix):] # drop prefix
          review_count += 1
        #   print(review_count)
          if review_count >= max_reviews:
            #   raise StopIteration
            return
          yield tokenizer(line) # TODO remove punctuation
          # line = line.strip().lower().split() 
  vocab = build_vocab_from_iterator(yield_tokens(), specials=[UNKNOWN_TOKEN])
  vocab.set_default_index(vocab[UNKNOWN_TOKEN])
  return vocab

In [ ]:
vocab = create_vocab(RATEBEER_FILE, MAX_REVIEWS)

In [ ]:
# from google.colab import drive
# torch.save(vocab, 'vocab.pth')
# drive.mount('/drive')
# !cp vocab.pth '/drive/MyDrive/Colab Notebooks/1e100ibu'


Mounted at /drive


In [ ]:
from google.colab import drive
drive.mount('/drive')
vocab = torch.load('/drive/MyDrive/Colab Notebooks/1e100ibu/vocab.pth')

Mounted at /drive


##### Test `vocab`

In [ ]:
review = 'On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.'
ided_review = vocab.lookup_indices(tokenizer(review))
print(f'id-ed review: {ided_review}')
word_count = len(vocab.get_itos())
print(f'word count: {word_count}')
print(f'most common words: {vocab.get_itos()[:30]}')
# try whether we can get an id for unknown token (which will be probably 0 as it's index of unknown token)
print(f'{vocab.lookup_indices(["lubie", "hamburgefonsz"])}')
# TODO remove keep only n most common words

id-ed review: [8, 29, 24, 4, 11, 2, 13, 22, 1, 28, 3, 63, 5, 135, 50, 53, 110, 3, 62, 54, 19, 10, 3, 52, 51, 34, 36, 12, 1, 38, 194, 86, 5, 109, 113, 132, 1, 92, 7, 86, 2, 6, 60, 5, 3, 17, 6, 70, 65, 1, 16, 235, 243, 84, 121, 4, 55, 73, 101, 16, 44, 14, 49, 23, 45, 3, 17, 6, 39, 10, 3, 48, 227, 23, 21, 1, 6, 33, 47, 8, 4, 69, 1]
word count: 247
most common words: ['<unk>', '.', ',', 'a', 'the', 'and', 'light', 'of', 'on', 'was', 'with', 'springfield', 'head', 'pa', 'this', 'aroma', 'i', 'very', 'as', 'color', 'had', 'it', 'location', 'to', 'at', 'beer', 'in', 'me', 'poured', 'tap']
[0, 0]


Test gradients

In [ ]:
# values = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)
# l2_loss = values.square().sum()
# # l2_loss.zero_grad()
# l2_loss.backward()

# print(values.grad)

tensor([2., 4., 6.])


## yyyy

In [ ]:
aspects = ['appearance', 'aroma', 'palate', 'taste', 'overall']
aspect_count = len(aspects)

In [ ]:
# theta = torch.empty(word_count, aspect_count, requires_grad=True)
theta = torch.rand((word_count, aspect_count), requires_grad=True)
# if dev == 'cuda':
#     theta = theta
with torch.no_grad():
    theta *= 0.9
#   torch.nn.init.kaiming_uniform_(theta)
  # enforce 1 initialization on aspect name (page 4)
    aspect_ids = vocab.lookup_indices(aspects)
    theta[aspect_ids, :] = 1

# phi = torch.rand((word_count, , aspect_count), requires_grad=True)

aspect_rating_count = [6, 11, 6, 11, 21]
phis = [torch.rand((word_count, aspect_rating_count[i]), requires_grad=True) for i in range(aspect_count)]

# if dev == 'cuda':
#     phis = [phi.cuda() for phi in phis]

with torch.no_grad():
    # normalize that sum across all words is 1 for a given aspect (eq. 7)
    phis = [phi / phi.sum(dim=0) for phi in phis]
for phi in phis:
    phi.requires_grad = True


In [ ]:
def review_text2ids(review_text: str):
  return vocab.lookup_indices(tokenizer(review_text))

# def review_text2weights(review_text: str):
#   ids = review_text2ids(review_text)
#   thetas = theta[ids]
#   phis   = phi[ids]
#   return thetas, phis

# def review_likelihood(review_text: str):
#   theta_weight, phi_weight = 1.0, 1.0
#   thetas, phis = review_text2weights(review_text)
#   return torch.mean(
#       theta_weight * thetas +
#       phi_weight * phis
#   )

In [ ]:
review_likelihood('Ich trinke Bier gern')

In [ ]:
review_likelihood('Amazing hops and taste')

In [ ]:
review_likelihood('appearance aroma palate taste overall')

In [ ]:
print(theta)

In [ ]:
vocab.lookup_indices(aspects)

#### review example

In [ ]:
#     """
#     get review like this
    
#     beer/name: John Harvards Simcoe IPA
#     beer/beerId: 63836
#     beer/brewerId: 8481
#     beer/ABV: 5.4
#     beer/style: India Pale Ale &#40;IPA&#41;
#     review/appearance: 4/5
#     review/aroma: 6/10
#     review/palate: 3/5
#     review/taste: 6/10
#     review/overall: 13/20
#     review/time: 1157587200
#     review/profileName: hopdog
#     review/text: On tap at the Springfield, PA location. Poured a deep and cloudy orange (almost a copper) color with a small sized off white head. Aromas or oranges and all around citric. Tastes of oranges, light caramel and a very light grapefruit finish. I too would not believe the 80+ IBUs - I found this one to have a very light bitterness with a medium sweetness to it. Light lacing left on the glass.
    
#     return review text as list of sentences and aspect ratings
#     """

### Implementation of $(1)$

In [ ]:
# tokenizer = get_tokenizer('basic_english')

def yield_ided_review(dataset_path=RATEBEER_FILE, max_lines=MAX_REVIEWS):
# def yield_ided_review(dataset_path=RATEBEER_FILE, max_lines=1000000000):
    with io.open(dataset_path, encoding='utf-8') as f:
        counter = 0
        review_lines = []
        for line, _ in zip(f, range(max_lines)):
            if counter < 13:
                review_lines.append(line)
                counter += 1
            else:
                review_aspects_scores = review_lines[5:10]
                review_aspects_scores = list(map(lambda s: int(s.split(': ')[1].split('/')[0]), review_aspects_scores))
                
                # wywalamy review/text
                review_sentences = review_lines[12].lower()[len("review/text: "):]
                if line.startswith("UPDATED:"):
                    review_sentences = review_sentences[len("UPDATED: APR 29, 2008"):] # drop prefix
                
                review_sentences = review_sentences.split('.')
                review_sentences = list(filter(lambda l: len(l) != 0, (map(review_text2ids, review_sentences))))
                result = (review_sentences, review_aspects_scores)
                # raise BaseException
                yield (review_sentences, review_aspects_scores) # TODO remove punctuation
                # line = line.strip().split()
                counter = 0
                review_lines = []


In [ ]:
# ided_reviews = yield_ided_reviews(max_lines=int(2e6))

In [ ]:
def sentence_aspects_likelihood_theta(sen_ids):
    theta_theta = theta[sen_ids]
    theta_score = theta_theta
    return theta_score

def sentence_aspects_likelihood_phi(sen_ids):
    phi_scores = [phis[aspect_idx][sen_ids, :] for aspect_idx in range(aspect_count)]
    return phi_scores

# def sentence_aspects_likelihood(sen_ids):
#     theta_score = sentence_aspects_likelihood_theta(sen_ids)
#     phi_scores = sentence_aspects_likelihood_phi(sen_ids)
#     ic(theta_score, phi_scores)

    # score = torch.exp( + sentence_aspects_likelihood_phi(sen_ids, ratings))
    # return score / score.sum()


In [ ]:
EPOCH_COUNT = 5
exit = False
for _ in range(EPOCH_COUNT):
    if exit:
        break
    for i, (review_sentences, review_aspects_scores) in enumerate(yield_ided_review(max_lines=int(50))):
    # for i, (review_sentences, review_aspects_scores) in enumerate(yield_ided_review(max_lines=int(2e6))):
        if exit:
            break
        print(i)
        for sen_ids in review_sentences:
            if exit:
                break
            try:
                theta_scores = sentence_aspects_likelihood_theta(sen_ids)
                aspect_pred = int(torch.argmax(torch.nn.functional.softmax(theta_scores.sum(0), dim=-1)).item())

                ic(vocab.lookup_tokens(sen_ids))

                
                ic(aspect_pred, aspects[aspect_pred])

                # ic(theta_scores.sum(0).shape) # TODO remove me

                aspect_rating = review_aspects_scores[aspect_pred]
                # ic(aspect_rating)
                # print(sentence_aspects_likelihood_phi(sen_ids)[aspect_pred])
                # print(sentence_aspects_likelihood_phi(sen_ids)[aspect_pred][:, aspect_rating])
                # print(sentence_aspects_likelihood_phi(sen_ids)[aspect_pred][:, 4])
                phi_score = sentence_aspects_likelihood_phi(sen_ids)[aspect_pred][:, aspect_rating]
                # ic(phi_score)

                # raise BaseException

                ll = torch.log(theta_scores[:, aspect_pred] + phi_score).sum()
                # ic(ll)

                l2_regularization_loss = torch.nn.functional.mse_loss(input=theta, target=torch.zeros_like(theta), reduction='mean') + torch.nn.functional.mse_loss(input=theta, target=torch.zeros_like(theta), reduction='mean')

                # ic(l2_regularization_loss)
                
                # ic(torch.hstack(list(torch.square(phi).sum() for phi in phis)).sum())
                
                loss = -ll + l2_regularization_loss * 0.1
                loss.backward()
                # ic(theta.grad)
                if 0 == i % 100:
                    ic(ll)
                    ic(l2_regularization_loss)
                    ic(loss)
                    print()

                with torch.no_grad():
                    theta -= 0.00001 * theta.grad
                    phis = [phi - 0.00001 * phi.grad if phi.grad != None else phi for phi in phis]
                    theta.grad = torch.zeros_like(theta.grad)
                    for phi in phis: phi.grad = torch.zeros_like(phi)
            except KeyboardInterrupt:
                exit = True


ic| vocab.lookup_tokens(sen_ids): ['on', 'tap', 'at', 'the', 'springfield', ',', 'pa', 'location']
ic| aspect_pred: 0, aspects[aspect_pred]: 'appearance'


0


ic| ll: tensor(-6.6337, grad_fn=<SumBackward0>)
ic| l2_regularization_loss: tensor(0.5607, grad_fn=<AddBackward0>)
ic| loss: tensor(6.6897, grad_fn=<AddBackward0>)
ic| vocab.lookup_tokens(sen_ids): ['poured',
                                   'a',
                                   'deep',
                                   'and',
                                   'cloudy',
                                   'orange',
                                   '(',
                                   'almost',
                                   'a',
                                   'copper',
                                   ')',
                                   'color',
                                   'with',
                                   'a',
                                   'small',
                                   'sized',
                                   'off',
                                   'white',
                                   'head']
ic

| aspect_pred: 0, aspects[aspect_pred]: 'appearance'
ic| ll: tensor(-15.9699, grad_fn=<SumBackward0>)
ic| l2_regularization_loss: tensor(0.5607, grad_fn=<AddBackward0>)
ic| loss: tensor(16.0260, grad_fn=<AddBackward0>)
ic| vocab.lookup_tokens(sen_ids): ['aromas', 'or', 'oranges', 'and', 'all', 'around', 'citric']
ic| aspect_pred: 2, aspects[aspect_pred]: 'palate'
ic| ll: tensor(-5.3487,

 grad_fn=<SumBackward0>)
ic| l2_regularization_loss: tensor(0.5607, grad_fn=<AddBackward0>)
ic| loss: tensor(5.4048, grad_fn=<AddBackward0>)
ic| vocab.lookup_tokens(sen_ids): ['tastes',
                                   'of',
                                   'oranges',
                                   ',',
                                   'light',
                                   'caramel',
                                   'and',
                                   'a',
                                   'very',
                                   'light',
                                   'grapefruit',
                                   'finish']
ic| aspect_pred: 4, aspects[aspect_pred]: 'overall'


...

)
ic| l2_regularization_loss: tensor(0.5607, grad_fn=<AddBackward0>)
ic| loss: tensor(3.2668, grad_fn=<AddBackward0>)
ic| vocab.lookup_tokens(sen_ids): ['on', 'tap', 'at', 'the', 'john', 'harvards', 'in', 'springfield', 'pa']
ic| aspect_pred: 0, aspects[aspect_pred]: 'appearance'
ic| vocab.lookup_tokens(sen_ids): ['pours',
                                   'a',
                                   'ruby',
                                   'red',
                                   'amber',
                                   'with',



1


                                   'a',
                                   'medium',
                                   'off',
                                   'whie',
                                   'creamy',
                                   'head',
                                   'that',
                                   'left',
                                   'light',
                                   'lacing']
ic| aspect_pred: 0, aspects[aspect_pred]: 'appearance'
ic| vocab.lookup_tokens(sen_ids): ['aroma', 'of', 'orange', 'and', 'various', 'other', 'citrus']
ic| aspect_pred: 4, aspects[aspect_pred]: 'overall'
ic| vocab.lookup_tokens(sen_ids): ['a',
                                   'little',
                                   'light',
                                   'for',
                                   'what',
                                   'i',
                                   'was',
                                   'expecting',
                   

2


budvar',
                                   'cristal',
                                   'but',
                                   'this',
                                   'is',
                                   'exactly',
                                   'what',
                                   'i',
                                   'imagined',
                                   'it',
                                   'to',
                                   'be']
ic| aspect_pred: 0, aspects[aspect_pred]: 'appearance'
ic| vocab.lookup_tokens(sen_ids): ['a',
                                   'clean',
                                   'and',
                                   'refreshing',
                                   ',',
                                   'hoppy',
                                   'beer',
                                   ',',
                                   'med',
                                   'bodied',
                                   '